# RAG 실습: Web 데이터 로딩 & PromptTemplate 활용

이 노트북에서는 포켓몬스터 나무위키 페이지에서 데이터를 로드하여 RAG(검색 증강 생성) 실습을 진행합니다.  
PromptTemplate을 활용한 다양한 프롬프트 구성 방법도 함께 다룹니다.

---
**실습 목표**
- 웹에서 문서를 로드하고, 텍스트를 청크로 분할
- 임베딩 및 벡터 DB 구축
- 검색 기반 답변 생성(RAG) 체인 구성
- PromptTemplate과 LangChain Hub 프롬프트 활용법 이해
- RunnablePassthrough 등 체인 유틸리티 활용법 익히기

## 1. 환경 변수 불러오기

- OpenAI API 키 등 민감 정보는 .env 파일에 저장합니다.
- dotenv를 통해 환경 변수를 로드합니다.

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

## 2. 웹 문서 로딩 및 텍스트 분할

- WebBaseLoader로 나무위키 포켓몬 타입 문서를 불러옵니다.
- RecursiveCharacterTextSplitter로 문서를 일정 길이의 청크로 분할합니다.
- 분할된 청크는 이후 임베딩 및 검색에 활용됩니다.

In [2]:
from langchain_community.document_loaders import WebBaseLoader

url = "https://namu.wiki/w/%ED%8F%AC%EC%BC%93%EB%AA%AC%EC%8A%A4%ED%84%B0/%ED%83%80%EC%9E%85"
loader = WebBaseLoader(url)
docs = loader.load()
print(f"Loaded {len(docs)} document(s) from {url}")

USER_AGENT environment variable not set, consider setting it to identify your requests.


Loaded 1 document(s) from https://namu.wiki/w/%ED%8F%AC%EC%BC%93%EB%AA%AC%EC%8A%A4%ED%84%B0/%ED%83%80%EC%9E%85


In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)
chunks = text_splitter.split_documents(docs)
print(f"Split into {len(chunks)} chunks")

Split into 29 chunks


## 3. 임베딩 및 FAISS DB 구축

- 분할된 청크 데이터를 임베딩(벡터화)합니다.
- FAISS 벡터 데이터베이스에 저장하여 검색이 가능하도록 만듭니다.
- OpenAIEmbeddings를 사용하며, save_local로 DB를 파일로 저장할 수 있습니다.

In [4]:
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

database = FAISS.from_documents(
    documents=chunks,
    embedding=embedding
)
database.save_local(folder_path="../faiss_pokemon")

In [5]:
# Retriever 준비
# as_retriever()로 벡터 DB에서 유사한 문서를 검색하는 객체를 생성합니다.
retriever = database.as_retriever(search_kwargs={'k': 2})

question = "포켓몬 타입 상성에 대해 알려줘."
retrieved_docs = retriever.invoke(question)
for i, doc in enumerate(retrieved_docs, 1):
    print(f"[{i}] {doc.page_content[:200]}...\n")

[1] (개별 비판)사건 사고버그 · 오류 · 오역 · 에딧 포켓몬 · 에메랄드 루프 · 개별 사건 사고미디어 믹스슈퍼 뮤직 컬렉션 · 외전 게임 · 애니메이션 · 포켓몬스터 SPECIAL · 포켓몬 카드 게임 · 포켓프라 · POKÉDANCE · Project VOLTAGE팬덤포켓몬덕후 · 2차 창작 · 괴담 · 밈 · 게이머 · 2차 창작자이벤트Pokémon ...

[2] 1.5배 증가하지만[14], 특성이 적응력인 경우엔 2배로 증가한다. 이를 '자기속성 보정'이라고 하며, 흔히 자속보정이란 줄임말로 통한다.[15] 대표적으로 피카츄의 10만볼트가 있다. 이 자속보정 때문에 두 개의 복합 타입을 가진 포켓몬은 공격적인 역할을 수행하는 어태커로써 평가받을 때 좀 더 유리하다.또한 일부 타입의 경우, 특정 상태이상을 무시하거나...



## 4. PromptTemplate을 활용한 답변 생성

- PromptTemplate을 사용하면 context(검색 결과)와 question(질문)을 조합해 원하는 형태의 프롬프트를 쉽게 만들 수 있습니다.
- 아래 예시는 context와 question을 받아 이모티콘을 활용해 친근하게 답변하도록 LLM에 요청합니다.

In [6]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o', temperature=0)

rag_prompt_template = PromptTemplate(
    template="""
    아래는 포켓몬 타입에 대한 정보입니다.
    이모티콘을 사용해서 친근하게 답변하세요.
    {context}

    질문: {question}
    답변:
    """,
    input_variables=["context", "question"]
)

context = "\n\n".join([doc.page_content for doc in retrieved_docs])
prompt = rag_prompt_template.format(context=context, question=question)
response = llm.invoke(prompt)
print(response.content)

안녕! 😊 포켓몬 타입 상성에 대해 궁금하구나! 포켓몬 세계에서는 각 포켓몬이 특정 타입을 가지고 있고, 이 타입에 따라 서로 다른 상성 관계가 있어. 이 상성 관계는 배틀에서 매우 중요한 역할을 해! 🏆

1. **기본 상성**:
   - **효과가 굉장함 (2배 대미지)**: 예를 들어, 물 타입 기술은 불꽃 타입 포켓몬에게 2배의 대미지를 줘. 💧🔥
   - **효과가 별로임 (0.5배 대미지)**: 예를 들어, 풀 타입 기술은 불꽃 타입 포켓몬에게 0.5배의 대미지를 줘. 🌿🔥
   - **효과가 없음 (0배 대미지)**: 예를 들어, 전기 타입 기술은 땅 타입 포켓몬에게는 전혀 효과가 없어. ⚡️🌍

2. **복합 타입**:
   - 포켓몬은 두 가지 타입을 가질 수 있어. 이 경우, 두 타입의 상성을 모두 고려해야 해. 예를 들어, 물/비행 타입 포켓몬은 전기 타입 기술에 4배의 대미지를 받게 돼. 💧🕊️⚡️

3. **특별한 타입**:
   - 게임 시리즈에 따라 특별한 타입이 등장하기도 해. 예를 들어, ??? 타입이나 스텔라 타입 같은 것들이 있어. 하지만 메인 시리즈에서는 잘 등장하지 않아. 🤔

이런 상성 관계를 잘 활용하면 배틀에서 유리한 위치를 차지할 수 있어! 포켓몬 배틀을 즐기면서 다양한 타입 조합을 시도해 보길 바랄게! 🥳


## 5. LangChain Hub의 프롬프트 활용

- LangChain Hub에는 다양한 RAG(검색 증강 생성) 시나리오에 맞게 설계된 표준 프롬프트가 저장되어 있습니다.
- `"rlm/rag-prompt"`는 context(검색 결과)와 question(질문)을 받아, LLM이 참고 문맥을 활용해 답변하도록 유도하는 구조의 프롬프트입니다.
- 직접 프롬프트를 작성하지 않아도 검증된 템플릿을 바로 사용할 수 있습니다.
- hub.pull()로 프롬프트를 불러와 chain에 바로 연결할 수 있습니다.

In [7]:
from langchain import hub

hub_prompt = hub.pull("rlm/rag-prompt")
rag_chain = hub_prompt | llm
result = rag_chain.invoke({"context": context, "question": question})
print(result.content)

c:\AKIS_Docs\6. 교육\4. 사내강사양성\rag_demo\.venv\lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


포켓몬 타입 상성은 공격하는 기술의 타입과 방어하는 포켓몬의 타입에 따라 달라집니다. 상성은 크게 효과가 굉장함(대미지 두 배), 효과가 별로임(대미지 0.5배), 효과가 없음(공격 무효)로 나뉩니다. 예를 들어, 물 타입은 불 타입에 강하고, 불 타입은 풀 타입에 강한 상성 관계를 가집니다.


## 6. RunnablePassthrough로 Chain 구성

- RunnablePassthrough는 입력값을 가공하지 않고 그대로 다음 단계로 전달할 때 사용하는 LangChain 유틸리티입니다.
- 여러 입력값 중 일부만 그대로 다음 runnable에 넘기고 싶을 때 활용할 수 있습니다.
- 아래 예시는 context는 retriever에서, question은 그대로 넘겨서 프롬프트에 전달합니다.

In [8]:
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | hub_prompt
    | llm
)
result = rag_chain.invoke(question)
print(result.content)

포켓몬 타입 상성은 공격하는 기술의 타입과 방어하는 포켓몬의 타입에 따라 달라집니다. 상성은 크게 네 가지로 나뉘며, 효과가 굉장한 경우(대미지 2배), 효과가 별로인 경우(대미지 0.5배), 효과가 없는 경우(공격 무효), 그리고 상성 관계가 없는 경우(대미지 보정 없음)로 구분됩니다. 각 타입은 서로 다른 상성을 가지며, 이를 통해 전략적인 전투가 가능합니다.
